In [1]:
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import matplotlib.image as mpimg
from skimage.transform import resize

In [4]:

  
  
  
#  negative class - Lung images with Pneumonia

X_train_neg = []
mypath = "data/train/PNEUMONIA/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for im_name in onlyfiles:
  if ".jpeg" in im_name:
    img = mpimg.imread(mypath+im_name)
    img = np.array(img)
    img = resize(img, (256,256))
    X_train_neg.append(img)

    
    
    
# negative labels are zero's

y_train_neg = []
for i in range(len(X_train_neg)):
  y_train_neg.append(1)
  
  
  
  
#  Some images has three channels, which are the duplicating the grayscale channel, 
#  so remove additional channels

for i in range(len(X_train_neg)):
  if len(X_train_neg[i].shape) > 2:
    X_train_neg[i] = X_train_neg[i].reshape(-1,256,256)[0]

    
#  Each list contains numpy arrays of with equal dimensions except first, 
#  so stack it to one numpy matrix

a = np.stack( X_train_pos, axis=0 )
b = np.stack( X_train_neg, axis=0 )
c = np.stack( y_train_pos, axis=0 )
d = np.stack( y_train_neg, axis=0 )

# X train file is concatenated positve and negative X classes
X_train = np.concatenate((a, b))
# the same with Y train file
y_train = np.concatenate((c, d))

# dereference lists which are no longer needed 
X_train_pos = None
X_train_neg = None
y_train_pos = None
y_train_neg = None

# Force garbage collector call to free memory which is no longer used
import gc
gc.collect()

8

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

#  Randomly split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


#  In order to apply linear models, we need to squeeze each datapoint to one-dimensional array
X_train = X_train.reshape(-1, 256*256)
X_test = X_test.reshape(-1, 256*256)

In [6]:
from sklearn.metrics import f1_score

# resize to 1D array to apply linear models

X_train = X_train.reshape(-1, 256*256)
X_test = X_test.reshape(-1, 256*256)


clf = LogisticRegression(solver='liblinear')
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)



print(accuracy)
print(auc)
print(f1)

/anaconda2/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9463414634146341
0.9325790910203876
0.9622641509433962


In [9]:
test = []
mypath = "data/test/"
for i in range(1757):
    img = mpimg.imread(mypath+'%d.jpeg' % (i+1))
    img = np.array(img)
    img = resize(img, (256,256))
    test.append(img)

for i in range(len(test)):
    if len(test[i].shape) > 2:
        test[i] = test[i].reshape(-1,256,256)[0]
    
x_test = np.stack( test, axis=0 )
x_test = x_test.reshape(-1, 256*256)
y_pred = clf.predict(x_test)

In [13]:
# preds = model.predict(x_test)
import pandas as pd
submission = pd.DataFrame({'id': range(len(y_pred)), 'pred': y_pred})
submission['id'] = submission['id'] + 1
submission.to_csv("data/ml_submission.csv", index=False, header=False)
submission.head()

,id,pred
0,1,0
1,2,0
2,3,1
3,4,1
4,5,1
